# Calculating all heuristics for a given tsp dataset

In [1]:
"""Third Party Import Statements"""
# Disable unused import not compatible with *.ipynb
# pylint: disable=W0611
import threading
import time
from networkx.exception import NetworkXError
from IPython.display import clear_output

from src import heuristics
from src import fileops as fo





In [22]:
HEURISTICS_FOLDER_PATH = 'XXdata/test'
TSP_SET_BASE_PATH = 'data/tspset1/tsp_'

In [20]:
def execute_mst(heuristic_name, function, instance, data, path):
    """
        Call the heuristics.mst function and save mst in given path
    """
    print(f'    COMPUTING:   {heuristic_name}')
    duration = time.time()
    mst, tree = function(instance)
    mst = int(mst)
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {mst}   ---   TIME: {duration}')
    data.append(mst)
    fo.save_as_json(data, f'{HEURISTICS_FOLDER_PATH}/{path}.json')
    return tree

# here we exceptionally allow 6 arguments for function
# pylint: disable=R0913
def execute_tree_heuristic(heuristic_name, function, instance, tree, data, path):
    """
        Call a given tree heuristic function. For eficiency mst can be passed as nx.Graph()
    """
    print(f'    COMPUTING:   {heuristic_name}   ---   ')
    duration = time.time()
    result = -1
    try:
        result = function(instance, tree)
        result = int(result)
    except NetworkXError:
        print('An error with a tree based heuristic occured')
        result = -1
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {result}   ---   TIME: {duration}')
    data.append(result)
    fo.save_as_json(data, f'{HEURISTICS_FOLDER_PATH}/{path}.json')

def execute_heuristic(heuristic_name, function, instance, data, path):
    """
        Execute a given TSP heuristic and save the result in a file
    """
    print(f'    COMPUTING:   {heuristic_name}   ---   ')
    duration = time.time()
    result = function(instance)
    result = int(result)
    duration = time.time() - duration
    print(f'    DONE:        {heuristic_name} : {result}   ---   TIME: {duration}')
    data.append(result)
    fo.save_as_json(data, f'{HEURISTICS_FOLDER_PATH}/{path}.json')

def update_heu(heu_data: dict):
    """
        update the given dictionary to be up to date with the data saved on the disc 
    """
    for key in heu_data.keys():
        heu_data[key] = fo.load_from_json(f'{HEURISTICS_FOLDER_PATH}/{key}.json')
    return heu_data

In [15]:
# update to represent actual
heu = {
    'mst':[],
    'christo': [],
    'onetree': [],
    'fi': [],
    'greedy': [],
    'mstheu': [],
    'ni': [],
    'nn': [],
    'opt': [],
    'ri': []
}

In [17]:
instances = 2000
start_time = time.time()
for i in range(instances):
    instance = fo.load_from_json(f'{TSP_SET_BASE_PATH}{i}_sol.json')
    dimension = instance['dimension']
    progress = ((i+1)/instances)*100
    passed_time = time.time() - start_time
    estimated_time = (passed_time / (i+1)) * (instances-i)
    print(f'PROCESSING INSTANCE {i+1} OF {instances}')
    print(f'PROGRESS: {progress}% \nINSTANCE SIZE: {dimension}')
    print(f'PASSED TIME: {passed_time}')
    print(f'ESTIMATED TIME LEFT {}')

    # first we need the mst
    tree = execute_mst('MINIMUM SPANNING TREE          ',
                       heuristics.mst,
                       instance, heu['mst'],
                       'mst')

   # in multithreading we compute mst dependend heuristics
    christo_thread = threading.Thread(
        target=execute_tree_heuristic,
        args=('CHRISTOFIDES                   ',
              heuristics.christo, instance,
              tree, heu['christo'],
              'christo'))
    
    onetree_thread = threading.Thread(
        target=execute_tree_heuristic,
        args=('1-TREE                         ',
              heuristics.onetree,
              instance,
              tree, heu['onetree'],
              'onetree'))
    
    mstheu_thread = threading.Thread(
        target=execute_tree_heuristic,
        args=('MINIMUM SPANNING TREE HEURISTIC',
              heuristics.mstheu,
              instance,
              tree, heu['mstheu'],
              'mstheu'))

    # and all other heuristics
    fi_thread = threading.Thread(
        target=execute_heuristic,
        args=('FARTHEST INSERTION             ',
              heuristics.fi,
              instance,  heu['fi'],
              'fi'))
    
    greedy_thread = threading.Thread(
        target=execute_heuristic, 
        args=('GREEDY HEURISTIC               ',
              heuristics.greedy,
              instance,  heu['greedy'],
              'greedy'))
    
    ni_thread = threading.Thread(
        target=execute_heuristic,
        args=('NEAREST INSERTION              ',
              heuristics.ni, instance,
              heu['ni'],
              'ni'))
    
    nn_thread = threading.Thread(
        target=execute_heuristic,
        args=('NEAREST NEIGHBOR               ',
              heuristics.nn,
              instance,  heu['nn'],
              'nn'))
    
    opt_thread = threading.Thread(
        target=execute_heuristic,
        args=('OPTIMAL TOUR                   ',
              heuristics.opt,
              instance, heu['opt'],
              'opt'))
    
    ri_thread = threading.Thread(
        target=execute_heuristic,
        args=('RANDOM INSERTION               ',
              heuristics.ri,
              instance, heu['ri'],
              'ri'))

    #defining threads
    threads = [
        fi_thread,
        ni_thread,
        ri_thread,
        christo_thread,
        onetree_thread,
        mstheu_thread,
        greedy_thread,
        nn_thread,
        opt_thread
    ]

    # starting threads
    for thread in threads:
        thread.start()

    # joining threads
    for thread in threads:
        thread.join()

    # update in memory datastructure with saved data
    heu = update_heu(heu)
    if i%2 != 0:
        clear_output()